<a href="https://colab.research.google.com/github/galall10/E-learning-/blob/main/Copy_of_NTI_Grad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

multi_instance_object_detection_challenge_path = kagglehub.competition_download('multi-instance-object-detection-challenge')

print('Data source import complete.')


Data source import complete.


In [ ]:
!pip install ultralytics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from pathlib import Path
import csv
import os
import torch
import cv2
from PIL import Image


In [ ]:
data_yaml = """
path: /kaggle/input/synthetic-2-real-object-detection-challenge-2/Synthetic to Real Object Detection Challenge 2

train:
  - /kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/clutter/train/images
  - /kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/couch_far_10/train/images
  - /kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/far_10_half_clutter/train/images
  - /kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/film_grain_10_half_clutter/train/images
  - /kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/large_plant_10/train/images
  - /kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/no_clutter_10/train/images
  - /kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/table_close_10/train/images
val:
  - /kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/clutter/val/images
  - /kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/couch_far_10/val/images
  - /kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/far_10_half_clutter/val/images
  - /kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/film_grain_10_half_clutter/val/images
  - /kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/large_plant_10/val/images
  - /kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/no_clutter_10/val/images
  - /kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/table_close_10/val/images

test: /kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/TestImages/images

nc: 1
names: ['soup can']
"""

# Save the configuration file
with open('data.yaml', 'w') as file:
    file.write(data_yaml)

print("Dataset configuration file created")


Dataset configuration file created


In [ ]:
# prompt: check if the photos are inn the created file

import glob

def check_images_in_paths(paths):
    found_images = {}
    for path_list in paths:
        if isinstance(path_list, str): # Handle the 'test' path which is a string
            path_list = [path_list]
        for path in path_list:
            # Use glob to find all files ending with common image extensions
            image_files = glob.glob(os.path.join(path, '*.[jJpPbBgGtT]*')) # Matches jpg, jpeg, png, bmp, gif, tiff
            found_images[path] = len(image_files) > 0
            if len(image_files) == 0:
                print(f"No images found in: {path}")
            else:
                print(f"Found {len(image_files)} images in: {path}")
    return found_images

# Extract the paths from the data_yaml string
import yaml
data_config = yaml.safe_load(data_yaml)

train_paths = data_config.get('train', [])
val_paths = data_config.get('val', [])
test_path = data_config.get('test')

# Check if images exist in each of the specified paths
print("Checking training image paths:")
check_images_in_paths(train_paths)

print("\nChecking validation image paths:")
check_images_in_paths(val_paths)

print("\nChecking test image path:")
check_images_in_paths([test_path]) # Pass test_path as a list for consistency with the function

Checking training image paths:
No images found in: /kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/clutter/train/images
No images found in: /kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/couch_far_10/train/images
No images found in: /kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/far_10_half_clutter/train/images
No images found in: /kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/film_grain_10_half_clutter/train/images
No images found in: /kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/large_plant_10/train/images
No images found in: /kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/no_clutter_10/train/images
No images found in: /kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/table_close_10/train/images

Checking validation image paths:
No images found in: /kaggle/input/multi-instance-object-detection-challenge/Starter_Data

{'/kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/TestImages/images': False}

In [ ]:
# prompt: concat the root path to make it work
multi_instance_object_detection_challenge_path = "/root/.cache/kagglehub/competitions/multi-instance-object-detection-challenge"

# Update the data_yaml string to use the correct root path
data_yaml = f"""
path: {multi_instance_object_detection_challenge_path}/Starter_Dataset

train:
  - {multi_instance_object_detection_challenge_path}/Starter_Dataset/clutter/train/images
  - {multi_instance_object_detection_challenge_path}/Starter_Dataset/couch_far_10/train/images
  - {multi_instance_object_detection_challenge_path}/Starter_Dataset/far_10_half_clutter/train/images
  - {multi_instance_object_detection_challenge_path}/Starter_Dataset/film_grain_10_half_clutter/train/images
  - {multi_instance_object_detection_challenge_path}/Starter_Dataset/large_plant_10/train/images
  - {multi_instance_object_detection_challenge_path}/Starter_Dataset/no_clutter_10/train/images
  - {multi_instance_object_detection_challenge_path}/Starter_Dataset/table_close_10/train/images
val:
  - {multi_instance_object_detection_challenge_path}/Starter_Dataset/clutter/val/images
  - {multi_instance_object_detection_challenge_path}/Starter_Dataset/couch_far_10/val/images
  - {multi_instance_object_detection_challenge_path}/Starter_Dataset/far_10_half_clutter/val/images
  - {multi_instance_object_detection_challenge_path}/Starter_Dataset/film_grain_10_half_clutter/val/images
  - {multi_instance_object_detection_challenge_path}/Starter_Dataset/large_plant_10/val/images
  - {multi_instance_object_detection_challenge_path}/Starter_Dataset/no_clutter_10/val/images
  - {multi_instance_object_detection_challenge_path}/Starter_Dataset/table_close_10/val/images

test: {multi_instance_object_detection_challenge_path}/Starter_Dataset/TestImages/images

nc: 1
names: ['soup can']
"""

# Save the updated configuration file
with open('data.yaml', 'w') as file:
    file.write(data_yaml)

print("Dataset configuration file created with updated paths")

# Extract the paths from the data_yaml string again (since it was updated)
data_config = yaml.safe_load(data_yaml)

train_paths = data_config.get('train', [])
val_paths = data_config.get('val', [])
test_path = data_config.get('test')

# Check if images exist in each of the specified paths
print("Checking training image paths:")
check_images_in_paths(train_paths)

print("\nChecking validation image paths:")
check_images_in_paths(val_paths)

print("\nChecking test image path:")
check_images_in_paths([test_path]) # Pass test_path as a list for consistency with the function

Dataset configuration file created with updated paths
Checking training image paths:
Found 8 images in: /root/.cache/kagglehub/competitions/multi-instance-object-detection-challenge/Starter_Dataset/clutter/train/images
Found 8 images in: /root/.cache/kagglehub/competitions/multi-instance-object-detection-challenge/Starter_Dataset/couch_far_10/train/images
Found 8 images in: /root/.cache/kagglehub/competitions/multi-instance-object-detection-challenge/Starter_Dataset/far_10_half_clutter/train/images
Found 8 images in: /root/.cache/kagglehub/competitions/multi-instance-object-detection-challenge/Starter_Dataset/film_grain_10_half_clutter/train/images
Found 8 images in: /root/.cache/kagglehub/competitions/multi-instance-object-detection-challenge/Starter_Dataset/large_plant_10/train/images
Found 8 images in: /root/.cache/kagglehub/competitions/multi-instance-object-detection-challenge/Starter_Dataset/no_clutter_10/train/images
Found 8 images in: /root/.cache/kagglehub/competitions/multi-i

{'/root/.cache/kagglehub/competitions/multi-instance-object-detection-challenge/Starter_Dataset/TestImages/images': True}

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8s.yaml").load("yolov8s.pt")  # you can also try yolov8n.yaml for even faster training

results = model.train(
    data="data.yaml",

    # Training performance
    epochs=200,
    batch=16,              # 16 fits well in 16GB for yolov8s + imgsz=640
    imgsz=640,             # Good balance of detail and memory use
    device=0,              # Use GPU

    # Optimizer
    optimizer='AdamW',
    lr0=0.001,
    weight_decay=0.0002,

    # Lighter augmentations
    mosaic=0.5,
    mixup=0.0,
    copy_paste=0.0,
    fliplr=0.5,
    flipud=0.0,
    erasing=0.1,

    # Advanced
    amp=True,              # Mixed precision for faster training

    # Logging and saving
    project='runs/detect',
    name='colab_t4_yolov8s',
    save_period=10,
    exist_ok=True,
    val=True
)

print("Training completed!")


100%|██████████| 21.5M/21.5M [00:00<00:00, 228MB/s]


Transferred 355/355 items from pretrained weights
Ultralytics 8.3.155 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.1, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.yaml, momentum=0.937, mosaic=0.5, multi_scale=False, name=colab_t4_yolov8s, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100, perspec

train: Scanning /root/.cache/kagglehub/competitions/multi-instance-object-detection-challenge/Starter_Dataset/clutter/train/labels.cache... 56 images, 1 backgrounds, 0 corrupt: 100%|██████████| 56/56 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 120.5±25.4 MB/s, size: 3806.8 KB)


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    154/200      3.96G     0.3156     0.2693     0.8134         27        640: 100%|██████████| 4/4 [00:01<00:00,  3.30it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  4.85it/s]

                   all         14         28      0.913      0.857      0.933      0.895



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    155/200      3.96G     0.2885      0.273     0.7974         51        640:  50%|█████     | 2/4 [00:00<00:00,  3.86it/s]

In [ ]:
# prompt: test this model and give me its mAP@50.

# Validate the trained model on the validation set
metrics = model.val()

# Access the mAP@50 metric
map50 = metrics.box.map50

print(f"mAP@50 on the validation set: {map50}")

Ultralytics 8.3.153 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 179.9±18.0 MB/s, size: 3956.7 KB)


val: Scanning /root/.cache/kagglehub/competitions/multi-instance-object-detection-challenge/Starter_Dataset/clutter/val/labels.cache... 14 images, 0 backgrounds, 0 corrupt: 100%|██████████| 14/14 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.40it/s]


                   all         14         28      0.927      0.911      0.957      0.902
Speed: 0.2ms preprocess, 7.9ms inference, 0.0ms loss, 24.5ms postprocess per image
Results saved to runs/detect/val
mAP@50 on the validation set: 0.9566764516129032


In [ ]:
import glob
import os
from ultralytics import YOLO

# Load model (assuming already trained and weights saved)
model = YOLO("runs/detect/colab_t4_yolov8s/weights/best.pt")

# Get all image paths in the test set
test_images = glob.glob(os.path.join(test_path, "*.*"))
print(f"Found {len(test_images)} test images.")

# Define chunk size (adjust to your memory; 10–20 is safe for T4 + 12GB)
chunk_size = 20

# Create a results folder
output_dir = "runs/detect/chunked_test"
os.makedirs(output_dir, exist_ok=True)

# Predict in chunks
for i in range(0, len(test_images), chunk_size):
    chunk = test_images[i:i + chunk_size]
    print(f"\n🔄 Running inference on images {i} to {i + len(chunk) - 1}")

    model.predict(
        source=chunk,
        conf=0.25,
        iou=0.45,
        imgsz=640,
        device=0,
        save=True,
        project="runs/detect",
        name="chunked_test",
        exist_ok=True
    )

print("\n✅ All chunks processed! Check runs/detect/chunked_test for results.")


ModuleNotFoundError: No module named 'ultralytics'

In [ ]:
# prompt: make the same predicate but on the /content/last.pt it is not testing

# Load the specific model
model = YOLO("/content/drive/MyDrive/saved_yolov8s_model.pt")

# Define the test path from the data_config
test_path = data_config.get('test')

# Get all image paths in the test set
test_images = glob.glob(os.path.join(test_path, "*.*"))
print(f"Found {len(test_images)} test images for prediction with /content/last.pt.")

# Define chunk size (adjust to your memory; 10–20 is safe for T4 + 12GB)
chunk_size = 20

# Create a results folder
output_dir = "runs/detect/last_pt_test"
os.makedirs(output_dir, exist_ok=True)

# Predict in chunks
for i in range(0, len(test_images), chunk_size):
    chunk = test_images[i:i + chunk_size]
    print(f"\n🔄 Running inference on images {i} to {i + len(chunk) - 1} using /content/last.pt")

    model.predict(
        source=chunk,
        conf=0.25,
        iou=0.45,
        imgsz=640,
        device=0,
        save=True,
        project="runs/detect",
        name="last_pt_test", # Unique name for this test run
        exist_ok=True
    )

print("\n✅ All chunks processed with /content/last.pt! Check runs/detect/last_pt_test for results.")



Found 179 test images for prediction with /content/last.pt.

🔄 Running inference on images 0 to 19 using /content/last.pt
Ultralytics 8.3.154 🚀 Python-3.11.13 torch-2.6.0+cu124 


ValueError: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: None
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.


In [ ]:
# prompt: create  Submission Format
# To participate, submit your csv file with predictions in YOLO format.
# Each test image should have a corresponding "image_id" and a "prediction_string" with predicted bounding boxes.
# CSV File Format
# Each prediction should contain predictions in YOLO format:
# "image_id", "prediction string(object class class_confidence
# x_center y_center width height)".
# image_id → image name without the .jpg/.png extension
# Object class → always 0 for soup in this competition.
# confidence → Prediction confidence (between 0 and 1).
# x_center, y_center, width, height → Coordinates normalized between 0 and 1.
# Example (0002.png for a single detected object)
# "0002","0 0.9435517191886902 0.874369642469618 0.30810502370198567 0.25126071506076386 0.25204394658406576"

# Prepare submission file
submission_df = pd.DataFrame(columns=['image_id', 'prediction_string'])

# Iterate through the test images and their corresponding prediction results
# The results are saved in the 'runs/detect/chunked_test' directory
# Each image will have its own folder containing the results, including labels
results_dir = "runs/detect/chunked_test"
image_files = glob.glob(os.path.join(test_path, "*.*"))

print("Generating submission file...")

for image_path in image_files:
    image_id = Path(image_path).stem
    prediction_string = ""

    # The label files from YOLO prediction are saved in a 'labels' subdirectory within the image's result folder
    # The structure is typically runs/detect/chunked_test/image_id/labels/image_id.txt
    label_file = os.path.join(results_dir, image_id, "labels", f"{image_id}.txt")

    if os.path.exists(label_file):
        with open(label_file, 'r') as f:
            lines = f.readlines()
            # Each line in the label file is in the format: class confidence x_center y_center width height
            # We need to convert this to the required submission format: 0 confidence x_center y_center width height
            # Since the competition requires class 0 for soup, we replace the actual class (which should be 0 from training) with 0.
            predictions = []
            for line in lines:
                parts = line.strip().split()
                if len(parts) == 5: # Ensure we have all 5 parts (class, confidence, x, y, w, h)
                    # The label file already contains normalized coordinates
                    # We need to add the class (which is always 0 for soup) and the confidence.
                    # The parts are: [class, x_center, y_center, width, height] from YOLO's txt output
                    # We need to get the confidence from the prediction result object if possible,
                    # but the saved label files in this format don't directly include confidence.
                    # A common approach is to re-process the model.predict results to get confidence.
                    # However, given the constraint to use the existing code context,
                    # and assuming the label file contains the correct format from YOLO (which *can* be configured to include confidence),
                    # let's assume the parts are [class, x_center, y_center, width, height] and we need the confidence.
                    # Since the prompt mentions "class_confidence x_center y_center width height" in the example,
                    # let's adapt to that and assume the saved label file is actually class x_center y_center width height.
                    # This means we need to get the confidence from the original prediction results.
                    # Re-running prediction to get results objects is more reliable for confidence.

                    # Let's adjust and re-run prediction on individual images to get the result objects
                    # to extract confidence correctly. This is more robust.
                    # For a large number of test images, we'd still do this in chunks.

                    # To avoid re-running prediction for the entire test set here,
                    # let's assume, for the sake of generating the submission format based on the provided label files,
                    # that the label file *does* have the confidence. This is a limitation based on the typical YOLOv8 label file format saved after 'save=True'.
                    # A better approach would be to process the `results` object from `model.predict` directly.

                    # Let's assume the format in the label file is 'class x_c y_c w h' and we will need to insert a placeholder confidence
                    # or, more accurately, re-run prediction to get actual confidence.
                    # Since we *already* ran `model.predict` with `save=True`, the bounding box coordinates are saved.
                    # The confidence is not in the saved .txt label files by default.
                    # We *must* get the confidence from the original prediction `results` objects.

                    # Let's refactor to process the prediction results directly instead of parsing saved label files.
                    pass # We will rewrite this loop below

    # Append an empty row for images with no detections (as required by some competitions)
    # Or, based on the example, if there are no detections, the prediction_string is empty.
    # Let's stick to the example format.

# --- New approach: Process prediction results directly ---
submission_data = []

print("Processing prediction results for submission...")

# Re-run prediction on test images to get result objects
# We need to do this again because the saved labels don't have confidence.
# We'll process chunk by chunk to manage memory.

test_images_paths = glob.glob(os.path.join(test_path, "*.*"))
chunk_size = 20 # Use the same chunk size

for i in range(0, len(test_images_paths), chunk_size):
    chunk = test_images_paths[i:i + chunk_size]
    print(f"\nProcessing chunk {int(i/chunk_size) + 1}/{(len(test_images_paths) + chunk_size - 1) // chunk_size}")

    results = model(chunk) # Run prediction on the chunk

    for j, result in enumerate(results):
        image_path = chunk[j]
        image_id = Path(image_path).stem
        prediction_string = ""

        if result.boxes: # Check if any boxes were detected
            predictions = []
            # Iterate through detected boxes in the result object
            for box in result.boxes:
                confidence = box.conf.item() # Get confidence as a standard Python float
                # Coordinates are already normalized in result.boxes
                x_center, y_center, width, height = box.xywhn[0].tolist() # Get normalized xywh as a list

                # Format: object class confidence x_center y_center width height
                # Object class is always 0 for soup
                prediction = f"0 {confidence:.6f} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
                predictions.append(prediction)

            prediction_string = " ".join(predictions)

        submission_data.append({'image_id': image_id, 'prediction_string': prediction_string})

# Create the submission DataFrame
submission_df = pd.DataFrame(submission_data)

# Save the submission file
submission_df.to_csv('submission.csv', index=False)

print("\nSubmission file 'submission.csv' created successfully.")
print(submission_df.head())

Generating submission file...
Processing prediction results for submission...

Processing chunk 1/9

0: 640x640 2 soup cans, 11.7ms
1: 640x640 2 soup cans, 11.7ms
2: 640x640 3 soup cans, 11.7ms
3: 640x640 2 soup cans, 11.7ms
4: 640x640 1 soup can, 11.7ms
5: 640x640 3 soup cans, 11.7ms
6: 640x640 2 soup cans, 11.7ms
7: 640x640 3 soup cans, 11.7ms
8: 640x640 3 soup cans, 11.7ms
9: 640x640 7 soup cans, 11.7ms
10: 640x640 3 soup cans, 11.7ms
11: 640x640 2 soup cans, 11.7ms
12: 640x640 2 soup cans, 11.7ms
13: 640x640 6 soup cans, 11.7ms
14: 640x640 3 soup cans, 11.7ms
15: 640x640 2 soup cans, 11.7ms
16: 640x640 1 soup can, 11.7ms
17: 640x640 2 soup cans, 11.7ms
18: 640x640 3 soup cans, 11.7ms
19: 640x640 12 soup cans, 11.7ms
Speed: 5.1ms preprocess, 11.7ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

Processing chunk 2/9

0: 640x640 3 soup cans, 10.4ms
1: 640x640 2 soup cans, 10.4ms
2: 640x640 2 soup cans, 10.4ms
3: 640x640 2 soup cans, 10.4ms
4: 640x640 4 soup cans, 10

In [ ]:
# prompt: make sure it does not contain null values

# Check for null values in the submission DataFrame
print("\nChecking for null values in the submission DataFrame:")
print(submission_df.isnull().sum())

# If there are nulls, you might need to investigate where they are coming from
# and decide how to handle them (e.g., fill with empty string if 'prediction_string' is null)
# In this specific case, based on the logic, 'prediction_string' should be an empty string if no detections
# or a non-empty string if detections exist. 'image_id' should never be null.
# However, let's add a check and fill if necessary.
if submission_df['prediction_string'].isnull().any():
  print("\nFound null values in 'prediction_string'. Filling with empty string.")
  submission_df['prediction_string'] = submission_df['prediction_string'].fillna('')

# Re-save the submission file after handling potential nulls
submission_df.to_csv('submission.csv', index=False)

print("\nSubmission file 'submission.csv' saved after null check.")
print(submission_df.head())


Checking for null values in the submission DataFrame:
image_id             0
prediction_string    0
dtype: int64

Submission file 'submission.csv' saved after null check.
   image_id                                  prediction_string
0  IMG_9757  0 0.897171 0.577790 0.301068 0.054396 0.103128...
1  IMG_9792  0 0.929153 0.583613 0.751287 0.079941 0.145574...
2  IMG_9607  0 0.949492 0.690762 0.471921 0.175357 0.169644...
3  IMG_9567  0 0.949058 0.763976 0.492270 0.137844 0.136842...
4  IMG_9769     0 0.830970 0.294869 0.577469 0.270854 0.676866


In [ ]:
# prompt: download this file on my machine

from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import os
from pathlib import Path
import csv
from ultralytics import YOLO
import glob # Added import for glob

def create_submission_from_results(
    model_path,
    test_images_folder,
    output_csv="submission.csv",
    confidence_threshold=0.25, # Use the same confidence as prediction
    allowed_extensions=(".jpg", ".png", ".jpeg")
):
    """
    Generate the submission CSV file from YOLO prediction results.

    Args:
        model_path (str): Path to the trained YOLO model weights (e.g., 'best.pt').
        test_images_folder (str): Path to the folder containing test images.
        output_csv (str): Path to save the output submission CSV file.
        confidence_threshold (float): Minimum confidence score to include a prediction.
        allowed_extensions (tuple): Tuple of allowed image file extensions.

    Returns:
        pd.DataFrame: The submission DataFrame.
    """
    model = YOLO(model_path)
    test_images_path = Path(test_images_folder)

    # Get all image paths in the test set
    test_image_files = [str(p) for p in test_images_path.glob("*") if p.suffix.lower() in allowed_extensions]
    print(f"Found {len(test_image_files)} test images.")

    submission_data = []
    chunk_size = 20 # Use a reasonable chunk size for prediction

    print("Processing prediction results for submission...")

    # Process prediction results in chunks
    for i in range(0, len(test_image_files), chunk_size):
        chunk = test_image_files[i:i + chunk_size]
        print(f"\nProcessing chunk {int(i/chunk_size) + 1}/{(len(test_image_files) + chunk_size - 1) // chunk_size}")

        results = model(chunk) # Run prediction on the chunk

        for j, result in enumerate(results):
            image_path = chunk[j]
            image_id = Path(image_path).stem
            prediction_string = ""

            if result.boxes: # Check if any boxes were detected
                predictions = []
                # Iterate through detected boxes in the result object
                for box in result.boxes:
                    confidence = box.conf.item() # Get confidence as a standard Python float

                    # Apply confidence threshold
                    if confidence >= confidence_threshold:
                        # Coordinates are already normalized in result.boxes
                        x_center, y_center, width, height = box.xywhn[0].tolist() # Get normalized xywh as a list

                        # Format: object class confidence x_center y_center width height
                        # Object class is always 0 for soup in this competition
                        prediction = f"0 {confidence:.6f} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
                        predictions.append(prediction)

                prediction_string = " ".join(predictions)

            submission_data.append({'image_id': image_id, 'prediction_string': prediction_string})

    # Create the submission DataFrame
    submission_df = pd.DataFrame(submission_data)

    # Sort by image_id
    submission_df = submission_df.sort_values('image_id')

    # Save the submission file
    submission_df.to_csv(output_csv, index=False)

    print(f"\nSubmission file '{output_csv}' created successfully.")
    print("Submission Preview:")
    print(submission_df.head())
    print(f"Submission file shape: {submission_df.shape}")

    return submission_df

# --- How to use the function ---
# Assuming your best model weights are saved at 'runs/detect/colab_t4_yolov8s/weights/best.pt'
# and your test images are at the path defined in data_yaml['test']
# You might need to adjust the paths based on your actual setup.

# First, make sure you have the 'test_path' variable defined from your data_yaml
import yaml
data_config = yaml.safe_load(data_yaml)
test_path = data_config.get('test')

# Call the function to create the submission file
submission_df = create_submission_from_results(
    model_path='runs/detect/colab_t4_yolov8s/weights/best.pt', # Adjust if your model path is different
    test_images_folder=test_path,
    output_csv='submission_from_results.csv' # Name your output file
)

# Optional: Check for null values again if needed
print("\nChecking for null values in the generated submission DataFrame:")
print(submission_df.isnull().sum())

Found 179 test images.
Processing prediction results for submission...

Processing chunk 1/9

0: 640x640 2 soup cans, 11.5ms
1: 640x640 2 soup cans, 11.5ms
2: 640x640 3 soup cans, 11.5ms
3: 640x640 2 soup cans, 11.5ms
4: 640x640 1 soup can, 11.5ms
5: 640x640 3 soup cans, 11.5ms
6: 640x640 2 soup cans, 11.5ms
7: 640x640 3 soup cans, 11.5ms
8: 640x640 3 soup cans, 11.5ms
9: 640x640 7 soup cans, 11.5ms
10: 640x640 3 soup cans, 11.5ms
11: 640x640 2 soup cans, 11.5ms
12: 640x640 2 soup cans, 11.5ms
13: 640x640 6 soup cans, 11.5ms
14: 640x640 3 soup cans, 11.5ms
15: 640x640 2 soup cans, 11.5ms
16: 640x640 1 soup can, 11.5ms
17: 640x640 2 soup cans, 11.5ms
18: 640x640 3 soup cans, 11.5ms
19: 640x640 12 soup cans, 11.5ms
Speed: 4.8ms preprocess, 11.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

Processing chunk 2/9

0: 640x640 3 soup cans, 11.2ms
1: 640x640 2 soup cans, 11.2ms
2: 640x640 2 soup cans, 11.2ms
3: 640x640 2 soup cans, 11.2ms
4: 640x640 4 soup cans, 11.2ms
5:

In [ ]:
# prompt: download the submission from results file

files.download('submission_from_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
model_path = "/content/drive/MyDrive/saved_yolov8s_model.pt"

In [ ]:
from ultralytics import YOLO

saved_model = YOLO(model_path)


In [ ]:
test_data_path = "/kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/TestImages/images"


In [ ]:
# 📝 Step 6: Optimized Label Generation
# ===============================================

print("📝 Generating optimized prediction labels...")

# Create output directory
output_dir = "/content/submission"
os.makedirs(output_dir, exist_ok=True)

# 🎯 Optimization parameters for multi-instance detection
conf_threshold = 0.25            # Lower confidence threshold
iou_threshold = 0.45             # Non-Max Suppression (NMS) threshold
max_detections_per_image = 3     # Max number of detections per image

# Process all test images
for img_path in Path(test_data_path).glob("*"):
    if img_path.suffix.lower() not in ['.jpg', '.jpeg', '.png']:
        continue

    # 🔥 Run prediction using optimized parameters
    results = saved_model.predict(
        img_path,
        conf=conf_threshold,
        iou=iou_threshold,
        agnostic_nms=True,
        max_det=max_detections_per_image,
        augment=True,           # Enable test-time augmentation
        verbose=False
    )

    # Output label file
    output_txt = Path(output_dir) / f"{img_path.stem}.txt"

    with open(output_txt, "w") as f:
        for result in results:
            img_height, img_width = result.orig_shape
            boxes = result.boxes.data

            if boxes is None or len(boxes) == 0:
                continue

            # 🎯 For multi-instance detection: sort by confidence, keep top 2 detections
            sorted_boxes = sorted(boxes.tolist(), key=lambda x: x[4], reverse=True)[:2]

            for box in sorted_boxes:
                x1, y1, x2, y2, conf, cls_id = box

                # Skip boxes with confidence below threshold
                if conf < conf_threshold:
                    continue

                # Convert to YOLO format (normalized)
                x_center = ((x1 + x2) / 2) / img_width
                y_center = ((y1 + y2) / 2) / img_height
                width = (x2 - x1) / img_width
                height = (y2 - y1) / img_height

                # Save in format: class_id confidence x_center y_center width height
                f.write(f"{int(cls_id)} {conf:.6f} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

print(f"✅ All prediction labels saved to: {output_dir}")


📝 Generating optimized prediction labels...
✅ All prediction labels saved to: /content/submission


In [ ]:
import pandas as pd
from pathlib import Path
import csv

print("📊 Generating the final submission file...")

def create_optimized_submission(
    preds_folder: str = "/content/submission",
    output_csv: str = "/kaggle/working/optimized_submission.csv",
    test_images_folder: str = "/kaggle/input/multi-instance-object-detection-challenge/Starter_Dataset/TestImages/images",
    allowed_extensions: tuple = (".jpg", ".png", ".jpeg")
):
    """
    Generate a submission CSV file from prediction text files.

    Args:
        preds_folder (str): Folder containing YOLO prediction .txt files.
        output_csv (str): Path to save the final submission CSV.
        test_images_folder (str): Folder containing the test images.
        allowed_extensions (tuple): Allowed image file types.

    Returns:
        pd.DataFrame: The final submission DataFrame.
    """
    preds_path = Path(preds_folder)
    test_images_path = Path(test_images_folder)

    # Get all test image names without extension
    test_images = {p.stem for p in test_images_path.glob("*") if p.suffix.lower() in allowed_extensions}

    predictions = []
    predicted_images = set()
    stats = {"with_predictions": 0, "no_predictions": 0}

    # Read predictions from .txt files
    for txt_file in preds_path.glob("*.txt"):
        image_id = txt_file.stem
        predicted_images.add(image_id)

        with open(txt_file, "r") as f:
            valid_lines = [line.strip() for line in f if len(line.strip().split()) == 6]

        if valid_lines:
            pred_str = " ".join(valid_lines)
            stats["with_predictions"] += 1
        else:
            pred_str = "no boxes"
            stats["no_predictions"] += 1

        predictions.append({"image_id": image_id, "prediction_string": pred_str})

    # Handle images that have no prediction file
    missing_images = test_images - predicted_images
    for image_id in missing_images:
        predictions.append({"image_id": image_id, "prediction_string": "no boxes"})
        stats["no_predictions"] += 1

    # Create and sort the submission DataFrame
    submission_df = pd.DataFrame(predictions)
    submission_df = submission_df.sort_values('image_id')
    submission_df.to_csv(output_csv, index=False, quoting=csv.QUOTE_MINIMAL)

    # Print stats
    print(f"📈 Submission Stats:")
    print(f"   Images with predictions: {stats['with_predictions']}")
    print(f"   Images without predictions: {stats['no_predictions']}")
    print(f"   Total images: {len(predictions)}")
    print(f"✅ Submission saved to: {output_csv}")

    return submission_df

# Generate the submission file
submission_df = create_optimized_submission()

# Preview the submission file
print("\n📋 Submission Preview:")
print(submission_df.head(10))
print(f"\nSubmission file shape: {submission_df.shape}")


📊 Generating the final submission file...


KeyError: 'image_id'

In [ ]:
# prompt: /content/submission_from_results.csv make it a data fram

submission_df = pd.read_csv('/content/submission_from_results.csv')
print(submission_df.head())
print(submission_df.info())
submission_df.shape

   image_id                                  prediction_string
0  IMG_9562  0 0.941095 0.282437 0.276353 0.077895 0.148537...
1  IMG_9563  0 0.880785 0.525480 0.579491 0.326818 0.345432...
2  IMG_9564  0 0.909203 0.481050 0.462901 0.228213 0.299315...
3  IMG_9565  0 0.853321 0.697713 0.189408 0.126730 0.127765...
4  IMG_9566  0 0.970180 0.358000 0.628993 0.126587 0.223958...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   image_id           179 non-null    object
 1   prediction_string  177 non-null    object
dtypes: object(2)
memory usage: 2.9+ KB
None


(179, 2)

In [ ]:
x = pd.read_csv("/content/submission.csv")
x

,image_id,prediction_string
0,IMG_9757,0 0.897171 0.577790 0.301068 0.054396 0.103128...
1,IMG_9792,0 0.929153 0.583613 0.751287 0.079941 0.145574...
2,IMG_9607,0 0.949492 0.690762 0.471921 0.175357 0.169644...
3,IMG_9567,0 0.949058 0.763976 0.492270 0.137844 0.136842...
4,IMG_9769,0 0.830970 0.294869 0.577469 0.270854 0.676866
...,...,...
174,IMG_9711,0 0.789559 0.395058 0.715779 0.057419 0.123681...
175,IMG_9759,0 0.912956 0.433836 0.385901 0.134016 0.276545...
176,IMG_9681,0 0.852444 0.418380 0.390726 0.121050 0.116531
177,IMG_9712,0 0.914191 0.328707 0.546428 0.065769 0.140778...


In [ ]:
# prompt: fill all the nulls in the submission with 0 and then download it

# Assuming 'submission_df' is already loaded and represents the submission data.
# Fill all null values in the DataFrame with 0
submission_df = submission_df.fillna(0)

# If the requirement is to fill empty strings in 'prediction_string' with 0 specifically,
# which is an unusual format for this type of submission, use the line below instead of the previous one.
# However, based on common competition formats, filling nulls with 0 and keeping empty strings as they are is more likely.
# If the intention is to replace the *text* "no boxes" or *empty strings* with the *string* "0", adjust accordingly.
# Let's stick to filling actual nulls with 0 as per the direct request "fill all the nulls... with 0".
# submission_df['prediction_string'] = submission_df['prediction_string'].replace('', '0') # Use this if empty string should become "0"

print("\nDataFrame after filling null values with 0:")
print(submission_df.isnull().sum())
print((submission_df['prediction_string'] == '').sum()) # Check if any empty strings remain (if they were not nulls)

# Save the updated submission DataFrame to a CSV file
submission_df.to_csv('submission.csv', index=False)

print("\nSubmission file 'submission.csv' updated after filling nulls with 0.")
print(submission_df.head())

# Download the updated submission file
files.download('submission.csv')



DataFrame after filling missing values:
image_id             0
prediction_string    0
dtype: int64
0
2


In [ ]:
# prompt: rechick it

submission_df = submission_df.fillna('') # Use empty string for missing predictions
print("\nDataFrame after filling missing values with empty string:")
print(submission_df.isnull().sum())
print((submission_df['prediction_string'] == '').sum())
print((submission_df['prediction_string'] == 'no boxes').sum())



DataFrame after filling missing values with empty string:
image_id             0
prediction_string    0
dtype: int64
0
2


In [ ]:
submission_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   image_id           179 non-null    object
 1   prediction_string  179 non-null    object
dtypes: object(2)
memory usage: 2.9+ KB


In [ ]:
submission_df

,image_id,prediction_string
0,IMG_9562,0 0.941095 0.282437 0.276353 0.077895 0.148537...
1,IMG_9563,0 0.880785 0.525480 0.579491 0.326818 0.345432...
2,IMG_9564,0 0.909203 0.481050 0.462901 0.228213 0.299315...
3,IMG_9565,0 0.853321 0.697713 0.189408 0.126730 0.127765...
4,IMG_9566,0 0.970180 0.358000 0.628993 0.126587 0.223958...
...,...,...
174,IMG_9788,0 0.949936 0.338021 0.488660 0.079062 0.165912...
175,IMG_9789,0 0.943173 0.307385 0.809598 0.077558 0.210937...
176,IMG_9790,0 0.922041 0.204219 0.405568 0.137591 0.202998...
177,IMG_9791,0 0.949616 0.478764 0.792190 0.091092 0.215915...
